In [1]:
'''
A script to do a sweep over defect density
'''
from munch import Munch
import numpy as np

########## Hamiltonian parameters ###########################################################################
Ham_params = Munch(
    t = 1,
    alpha = 1.25,
    mu = 0,
    defect_density = 'varying',
    J = 5,
    U = 5,
    beta = np.NaN,
    normalise = True #Whether the long range interaction should be normalised against the CDW or not.
)
print('Ham_params: ', ' '.join(f'{k}={v},' for k,v in Ham_params.items()))

########## Variable Hamiltonian parameters ###########################################################################
chain_exts = np.arange(0,2) #the number of times to extend the chain maximum is about 50 on CX1
N_steps = int(20e3) #the number of MCMC steps in each individual task
thin = 1

print(f'''
Tasks per chain: {chain_exts.size},
Each doing {N_steps} steps,
{chain_exts.size*N_steps} total chain length,
{chain_exts.size*N_steps // thin} samples,
''')

defect_densities = np.linspace(0, 1, 100)
Ns = np.logspace(np.log10(60), np.log10(270), 3, dtype = np.int64) // 10 * 10

from FKMC.montecarlo import Eigenspectrum_IPR_all
def loggerfactory(): return Eigenspectrum_IPR_all(bins = 2000, limit = 5)

structure_names = ['defect_densities',] #Ns and chain_exts is dealt with separately
structure_dimensions = [defect_densities,]


########## Batch Job Structure ###########################################################################
from itertools import product as cartesian_product

config_product = cartesian_product(*structure_dimensions)

#give information to the dispatch script
batch_params = Munch(
    total_jobs = np.prod([len(dim) for dim in structure_dimensions]),
    chain_exts = chain_exts,
    structure_names = structure_names, #names of each of the dimensions like ['Ts', 'Alphas']
    structure_dimensions = structure_dimensions, #the dimensions themselves like [np.linspace(0.1,5,100), np.linspace(0.1,2,100)]
    indices = (0, np.prod([len(dim) for dim in structure_dimensions])),
)
#bath_params_end_flag this is here to signal the end of the batch_params variable

########## Parameters particular to this job ################################################################

from itertools import product, islice
from pathlib import Path
import os
from time import time, sleep
import sys
import shutil
from FKMC.import_funcs import timefmt

print('Getting environment variables')
debug = (os.getenv('DEBUG', 'True') == 'True')
submit_dir = Path(os.getenv('SUBMIT_DIR', Path('~/HPC_data/test/').expanduser()))

job_id = int(os.getenv('JOB_ID', -1))
chain_id = int(os.getenv('CHAIN_ID', -1))
task_id = int(os.getenv('TASK_ID', 0))

print(f'job_id = {job_id}, chain_id = {chain_id}, task_id = {task_id}, submit_dir = {submit_dir}')

Is_todo = np.arange(len(Ns))
Ns_todo = Ns.copy()
logs = [None for _ in Ns]

filename = f'{task_id}_{chain_id}.npz'

if not debug and (submit_dir / 'data' / filename).exists():
    print(f'Job file {filename} already exists')
    print(f'Loading {filename} to retrieve the work')
    d = Munch(np.load(submit_dir / 'data' / filename, allow_pickle = True))
    logs = d['logs'][()]
    todo = (logs == None)
    Is_todo = Is_todo[todo]
    Ns_todo = Ns_todo[todo]
    print(f'Ns_todo = {Ns_todo}')
    


(defect_density,), = list(islice(config_product, task_id, task_id + 1))
print(f'defect_density = {defect_density}')
Ham_params.defect_density = defect_density

########## Set up debugging and sleep ################################################################

if debug: N_steps = 1000
    

##sleep if necessary
if not debug: 
    sleeptime = np.random.random() * (10)
    print(f'Waiting for {sleeptime:.0f} seconds to randomise the finish time')
    sleep(sleeptime)

########## A benchmark for spotting weird nodes ######################################################
from FKMC.general import tridiagonal_diagonalisation_benchmark
if not debug:
    benchmark_time = tridiagonal_diagonalisation_benchmark()
    print(f"Diagonalisation benchmark: {benchmark_time:.2f} seconds")

########## The actual simulation code ################################################################
from FKMC.disorder_model import FK_disorder_model
from numpy.random import default_rng

Ham_params['J_matrix'] = '...'
t0 = time()
rng = default_rng()

for i, N in zip(Is_todo, Ns_todo):
    logger = loggerfactory()
    states = rng.choice((0,1), size = (N_steps, N))
    
    t0 = time()
    logs[i] = FK_disorder_model(states, logger = logger, parameters = Ham_params, rng = rng)
    logs[i].time = time() - t0
    
    np.savez_compressed(filename, 
        Ns = Ns, parameters = Ham_params,
        structure_names = structure_names,
        structure_dimensions = structure_dimensions,        
        chain_id = chain_id,
        task_id = task_id,
        logs = logs, allow_pickle = True,
        desc = ''
        )
    
    shutil.copy(filename, submit_dir / 'data')

t = time() - t0
print(f'{t:.0f} seconds, final saving in {Path.cwd()} / {filename}')
print(f'Copying to {submit_dir}/data)')

print(f'''
Requested disorder realisations: {N_steps}
Time: {timefmt(t)} 
Estimated task runtime for 10,000 steps: {timefmt(t * 10_000 / N_steps)} 
''')
print('Done')

Ham_params:  t=1, alpha=1.25, mu=0, defect_density=varying, J=5, U=5, beta=nan, normalise=True,

Tasks per chain: 2,
Each doing 20000 steps,
40000 total chain length,
40000 samples,

Getting environment variables
job_id = -1, chain_id = -1, task_id = 0, submit_dir = /rds/general/user/tch14/home/HPC_data/test
defect_density = 0.0


ModuleNotFoundError: No module named 'FKMC.disorder_model'